# 🔄 Inferência Recorrente de Vendas - CVC Lojas

## 🎯 Objetivo
Execução periódica (Semanal/Mensal) para gerar novas previsões de vendas.
Este notebook não treina modelos. Ele carrega o modelo produtivo (`All-in-One`) e gera forecast baseando-se no histórico mais recente.

## ⚙️ Fluxo de Execução
1.  **Context Loading:** Carrega os últimos 90 dias de vendas (Janela de Contexto) do Data Lake.
2.  **Model Loading:** Baixa o modelo do Unity Catalog (`Usage: Production`).
3.  **Inference:** O Wrapper `UnifiedForecaster` recebe o contexto, normaliza, prevê e desnormaliza.
4.  **Persistence:** Salva os resultados na tabela `bip_vprevisao_lojas_futuro`.


In [0]:
# --- SETUP INICIAL ---
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.getcwd())

from src.validation.config import Config
from src.validation.data import DataIngestion
from datetime import timedelta, date
import mlflow
import pyspark.sql.functions as F
import pandas as pd

# Darts classes para Wrapper funcionar
from darts import TimeSeries

# Configs Spark
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")

In [0]:
# --- 1. DEFINIÇÃO DA JANELA DE CONTEXTO ---

# Simulando "Hoje" (em prod usar date.today())
today = date.today()

# Janela de Lookback: precisamos de histórico suficiente para os Lags do modelo (ex: 60-90 dias)
context_days = 90 
start_context = today - timedelta(days=context_days)

# Config Dinâmica
config = Config(spark)
config.DATA_START = start_context.strftime("%Y-%m-%d")
config.INGESTION_END = today.strftime("%Y-%m-%d")
config.SCHEMA = "cvc_pred"

print(f"📅 Data de Referência (Hoje): {today}")
print(f"🔎 Carregando contexto a partir de: {config.DATA_START}")

In [0]:
# --- 2. CARREGAMENTO DO CONTEXTO (SPARK) ---
# Reutilizamos a classe DataIngestion para garantir consistência nas features
ingestion = DataIngestion(spark, config)

print("   ⏳ Lendo dados históricos recentes...")
df_context_spark = ingestion.create_training_set()

# Filtro de Segurança e Seleção de Colunas
df_context_spark = df_context_spark.filter(
    F.col("DATA").between(config.DATA_START, config.INGESTION_END)
)

# Traz para Pandas (Driver) - Volume pequeno pois é só janela recente
df_context_pd = df_context_spark.toPandas()
df_context_pd['DATA'] = pd.to_datetime(df_context_pd['DATA'])

# Injeta parametro 'n' para o wrapper saber o horizonte desejado
FORECAST_HORIZON = 35
df_context_pd['n'] = FORECAST_HORIZON

print(f"   ✅ Contexto Carregado: {len(df_context_pd)} registros de vendas recentes.")
display(df_context_pd.head(5))

In [0]:
# --- 3. CARREGAMENTO DO MODELO ---

model_name = f"{config.CATALOG}.{config.SCHEMA}.cvc_lojas_forecast_production"
print(f"📥 Baixando modelo do Unity Catalog: {model_name}...")

# Em produção real, você usaria um Alias como "@Prod" ou a versão specifica
# modelo = mlflow.pyfunc.load_model(f"models:/{model_name}@Prod")
# Aqui carregamos a última versão logs
loaded_model = mlflow.pyfunc.load_model(f"models:/{model_name}/1") 
print("   ✅ Modelo Carregado!")

In [0]:
# --- 4. PREVISÃO (SCORING) ---
print("🔮 Gerando previsões futuras...")

# O método predict do nosso Wrapper já faz tudo:
# 1. Recebe o DF brudo com histórico
# 2. Reconstrói a TimeSeries
# 3. Aplica o Scaler salvo
# 4. Prevê
# 5. Inverte o Scaler
forecast_df = loaded_model.predict(df_context_pd)

# Enriquecimento final
forecast_df['DATA_REFERENCIA_EXECUCAO'] = today

print(f"   ✅ Previsões geradas: {len(forecast_df)} registros.")
display(forecast_df.head())

In [0]:
# --- 5. PERSISTÊNCIA (WRITE BACK) ---
output_table = f"{config.CATALOG}.{config.SCHEMA}.bip_vprevisao_lojas_futuro"

print(f"💾 Salvando resultados em: {output_table}")

try:
    (spark.createDataFrame(forecast_df)
     .write
     .format("delta")
     .mode("append") # Append histórico de previsões
     .option("mergeSchema", "true")
     .saveAsTable(output_table)
    )
    
    spark.sql(f"OPTIMIZE {output_table}")
    print("   ✨ Sucesso! Dados salvos e otimizados.")
except Exception as e:
    print(f"❌ Erro ao salvar: {e}")